In [ ]:
import wikipediaapi as wp
import re
import os
import nltk
from nltk import stem
from nltk.corpus import stopwords
import string
import json
import time

from src.etl.get_anames import retrieve
from src.libcode import txt_to_list, list_to_txt

In [ ]:
enwp = wp.Wikipedia("en")
anames = retrieve()

In [ ]:
pp_dir = "src/data/init/partisan_phrases/"
pp_txts = os.listdir(pp_dir)
score_dict = {}
for i in pp_txts:
    with open(pp_dir + i) as curtxt:
        for line in curtxt.readlines()[1:]:
            splt = line.split("|")
            score_dict[splt[0]] = float(splt[1].strip())

In [ ]:
# num_per_text = len(anames)//10
# wikitxts_dir = "src/data/temp/wiki_txts/"

# # If wiki texts folder does not exist make it
# if not os.path.exists(wikitxts_dir):
#     os.makedirs(wikitxts_dir)

# txtlst = []
# for ind, aname in enumerate(anames):
#     # Get the page text
# #     print(ind, aname)
#     curpg = enwp.page(aname)
#     curtit = curpg.title
#     curtxt = curpg.text
#     txtlst.append(curtit)
#     txtlst.append(curtxt)
#     # This ensures it saves into 10 txt files
#     if (ind+1) % num_per_text == 0:
#         print(ind+1)
#         curtxt_name = "art_pages" + str((ind+1)//num_per_text) + ".txt"
#         list_to_txt(wikitxts_dir+curtxt_name, txtlst)
#         txtlst = []
#         time.sleep(10)
        
# # Save last set of articles
# if len(txtlst) > 0:
#     curtxt_name = "art_pages10" + ".txt"
#     list_to_txt(wikitxts_dir+curtxt_name, txtlst)

In [ ]:
sorted(score_dict.items(), key=lambda item: item[1], reverse=False)

In [ ]:
# nltk.download("wordnet")
# nltk.download("stopwords")

stpwrds = stopwords.words("english")
porter = stem.PorterStemmer()

def string_score(strn, score_dict):
    # Lowercase, remove digits and doublespaces
    curstr = strn.lower().translate(str.maketrans('', '', string.punctuation))
    curstr = re.sub(r'[0-9]+', '', curstr)
    curstr = re.sub(r'\n', ' ', curstr)
    curstr = re.sub(r'  +', ' ', curstr)
    plst = []
    for word in curstr.split():
        # Check for stopwords
        if word not in stpwrds:
            
#             if len(word) < 4:
#                 print(word)
            
            # Stem the word
            pword = porter.stem(word)

    #         # Put the stemmed word in the reverse-stemming dictionary
    #         if pword not in stem_rev.keys():
    #             stem_rev[pword] = [word]
    #         elif word not in stem_rev[pword]:
    #             stem_rev[pword].append(word)
            
            plst.append(pword)
    curstrlen = len(plst)
    curstr = ' '.join(plst)

    absscore = 0
    sumscore = 0
    
    counts_dict = {}
    
    for key, value in score_dict.items():
        numoccurs = curstr.count(key)
        counts_dict[key] = (numoccurs, value)
        curscore = numoccurs*value
        absscore += abs(curscore)
        sumscore += curscore

    counts_dict = sorted(counts_dict.items(), key=lambda item: item[1], reverse=True)[:10]
    return (absscore, sumscore, curstrlen, counts_dict)

In [ ]:
wikitxts_dir = "src/data/temp/wiki_txts/"
wiki_txts = ["art_pages" + str(i) + ".txt" for i in range(1,11)]
namestat_dict = {}

cnt = 0
for txt in wiki_txts:
    print(0)
    txtlst = txt_to_list(wikitxts_dir + txt)
    print(len(txtlst))
    for item in txtlst:
        if cnt % 2 == 0:
            print(item)
            aname = item
        else:
            curres = string_score(item,score_dict)
            namestat_dict[aname] = curres
        cnt += 1

In [ ]:
namestat_dict